In [1]:
!ls /home/notebook/test/

Vishal	common	naarni_venv


In [2]:
import gc
import os
import sys
import platform
import logging
import argparse
from datetime import datetime, date, timedelta
import pendulum
import pandas as pd
import numpy as np

repo_path = '/home/notebook'
sys.path.append(os.path.join(repo_path, 'test'))


# from common.cpoall_aggregator import run_daily_tms_analysis
# Import necessary files and its respective functions
from common.db_operations import connect_to_trino, fetch_distinct_ids_for_day_trino,fetch_data_for_day_trino, write_df_to_iceberg, parse_arguments, get_target_date
from common.cpoall_aggregator import run_daily_tms_analysis,CORE_COLS

In [3]:
# conn = connect_to_trino()
# sql = f"DROP TABLE IF EXISTS adhoc.facts_dev.bcs_tms_sessions_v1"
# execute_query(conn, sql, return_results=False)
# conn.close()

In [ ]:
def run_tms_sessions_range_to_iceberg(
    conn,
    start_date: str,
    end_date: str,
    core_cols,
    df_mapping=None,
    source_schema: str = "facts_prod",
    source_table: str = "can_parsed_output_all",
    target_schema: str = "facts_dev",
    target_table: str = "bcs_tms_sessions_v1",
    partition_by=("date",),
    push_to_db: bool = True,
    collect_sessions: bool = False,
    ids=None,   # optional explicit list of device IDs (global filter)
):
    """
    Memory-safe range runner for TMS sessions.

    - Loops from start_date to end_date (inclusive), interpreted in IST.
    - For each day:
        * If `ids` is None:
            - discover the IDs that actually have data on this IST day
          else:
            - use the provided `ids` list as a filter
        * For each id in that day's list:
            - fetch_data_for_day_trino(conn, day, [id], ...)
            - run_daily_tms_analysis(...)
        * After all ids for that day:
            - concat all sessions for that day
            - optionally write that day's sessions to Iceberg
    """

    # --- normalize start_date / end_date to date objects ---
    if isinstance(start_date, str):
        start = datetime.strptime(start_date, "%Y-%m-%d").date()
    else:
        start = start_date

    if isinstance(end_date, str):
        end = datetime.strptime(end_date, "%Y-%m-%d").date()
    else:
        end = end_date

    if end < start:
        raise ValueError("end_date must be >= start_date")

    n_days = (end - start).days + 1

    # Optional global filter: if user passes ids, we’ll intersect with per-day ids
    global_ids_filter = None
    if ids is not None:
        global_ids_filter = set(str(x) for x in ids)

    logging.info(
        "🚀 TMS sessions range job: %s → %s (%d days)",
        start,
        end,
        n_days,
    )

    all_days_sessions = [] if collect_sessions else None

    day = start
    while day <= end:
        day_str = day.strftime("%Y-%m-%d")
        logging.info("📅 Processing IST day %s", day_str)

        # ------------------------------------------------------------------
        # Per-day ID discovery
        # ------------------------------------------------------------------
        if global_ids_filter is None:
            ids_for_day = fetch_distinct_ids_for_day_trino(
                conn,
                day,
                schema=source_schema,
                table=source_table,
            )
        else:
            # discover ids for day, then intersect with global filter
            day_ids_raw = fetch_distinct_ids_for_day_trino(
                conn,
                day,
                schema=source_schema,
                table=source_table,
            )
            ids_for_day = [vid for vid in day_ids_raw if vid in global_ids_filter]

        if not ids_for_day:
            logging.info("  ℹ️ No IDs with data on %s. Skipping entire day.", day_str)
            day += timedelta(days=1)
            continue

        logging.info(
            "  🔎 Found %d IDs with data on %s: %s",
            len(ids_for_day),
            day_str,
            ", ".join(ids_for_day),
        )

        # Collect all sessions for this day (across these IDs)
        day_sessions_list = []

        for vid in ids_for_day:
            logging.info("  🚍 Processing id=%s on %s", vid, day_str)

            # Fetch raw data for THIS (id, day)
            df_raw = fetch_data_for_day_trino(
                conn,          # connection
                day,           # date (IST calendar day)
                [vid],         # list of ids (single id)
                core_cols=core_cols,
                schema=source_schema,
                table=source_table,
            )

            if df_raw.empty:
                # This should be rare now, but keep it safe
                del df_raw
                gc.collect()
                continue

            # TMS analysis for this id/day
            daily = run_daily_tms_analysis(
                df_raw=df_raw,
                df_mapping=df_mapping,
            )
            sessions_id = daily["sessions"]

            if sessions_id.empty:
                del df_raw, daily, sessions_id
                gc.collect()
                continue

            # ensure 'date' is this IST day
            if "date" not in sessions_id.columns:
                sessions_id["date"] = day
            else:
                sessions_id["date"] = pd.to_datetime(sessions_id["date"]).dt.date
                sessions_id["date"] = day

            day_sessions_list.append(sessions_id)

            del df_raw, daily, sessions_id
            gc.collect()

        # --- after looping over all ids for this day ---
        if not day_sessions_list:
            logging.info("  ℹ️ No sessions built for %s across all IDs. Skipping DB write.", day_str)
            day += timedelta(days=1)
            continue

        day_sessions_df = pd.concat(day_sessions_list, ignore_index=True)
        logging.info(
            "  ✅ Built %d sessions total for %s (across %d IDs).",
            len(day_sessions_df),
            day_str,
            day_sessions_df["id"].nunique() if "id" in day_sessions_df.columns else -1,
        )

        # Single write per day
        if push_to_db:
            write_df_to_iceberg(
                conn=conn,
                df=day_sessions_df,
                schema=target_schema,
                table=target_table,
                partition_by=list(partition_by) if partition_by else None,
            )
            logging.info(
                "  💾 Inserted %d rows into %s.%s for %s",
                len(day_sessions_df),
                target_schema,
                target_table,
                day_str,
            )

        if collect_sessions:
            all_days_sessions.append(day_sessions_df.copy())

        del day_sessions_list, day_sessions_df
        gc.collect()

        day += timedelta(days=1)

    logging.info("✅ Completed TMS sessions range job: %s → %s", start, end)

    if collect_sessions:
        if all_days_sessions:
            return pd.concat(all_days_sessions, ignore_index=True)
        else:
            return pd.DataFrame()

    return None


In [ ]:
# conn = connect_to_trino()

# run_tms_sessions_range_to_iceberg(
#     conn=conn,
#     start_date="2025-09-01",
#     end_date="2025-09-03",
#     core_cols=CORE_COLS,
#     df_mapping=None,
#     source_schema="facts_prod",
#     source_table="can_parsed_output_all",
#     target_schema="facts_dev",
#     target_table="bcs_tms_sessions_v2",
#     push_to_db=True,
#     partition_by=("date",),
#     collect_sessions=False,   # pure pipeline mode
# )

# # quick sanity check
# # sessions_all.query("date == '2025-09-02' and id == '16'").head()

2025-12-11 22:25:45 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-11 22:25:45 - INFO - ✅ STEP 1: Connected to Trino
2025-12-11 22:25:45 - INFO - 🚀 TMS sessions range job: 2025-08-01 → 2025-08-03 (3 days)
2025-12-11 22:25:45 - INFO - 📅 Processing IST day 2025-08-01
2025-12-11 22:25:45 - INFO - 🔍 Fetching distinct ids for 2025-08-01 (UTC window 2025-07-31 18:30:00 → 2025-08-01 18:30:00)
2025-12-11 22:25:47 - INFO - 🔍 Found 11 ids with data on 2025-08-01
2025-12-11 22:25:47 - INFO -   🔎 Found 11 IDs with data on 2025-08-01: 11, 12, 13, 14, 15, 16, 18, 3, 6, 7, 9
2025-12-11 22:25:47 - INFO -   🚍 Processing id=11 on 2025-08-01
2025-12-11 22:25:51 - INFO -   🚍 Processing id=12 on 2025-08-01
2025-12-11 22:25:54 - INFO -   🚍 Processing id=13 on 2025-08-01
2025-12-11 22:25:58 - INFO -   🚍 Processing id=14 on 2025-08-01
2025-12-11 22:26:03 - INFO -   🚍 Processing id=15 on 2025-08-01
2025-12-11 22:26:07 - INFO -   🚍 Processing id=16 on 2025-08-01
2025-12-11 22:26:09 - INFO -   🚍 Processing id

In [ ]:
# ---- TMS main, mirroring energy_mileage main() ----
def main(
    start_date_str: str | None = None,
    end_date_str: str | None = None,
    source_schema: str = "facts_prod",
    source_table: str = "can_parsed_output_all",
    target_schema: str = "facts_dev",
    target_table: str = "bcs_tms_sessions_v1",
    push_to_db: bool = True,
    partition_by: tuple | None = ("date",),
    ids: list[str] | None = None,
):
    """
    Entry point for the BCS/TMS sessions pipeline.

    - If start_date_str and end_date_str are provided:
        process that inclusive date range (IST).
    - Otherwise:
        use CLI args via parse_arguments() + get_target_date()
        and process a single IST day (usually 'yesterday').
    """

    conn = connect_to_trino()
    if not conn:
        logging.critical("❌ Failed to establish a database connection. Exiting.")
        return

    try:
        # Decide date_range
        if start_date_str and end_date_str:
            logging.info(
                f"📅 Processing explicit date range: {start_date_str} to {end_date_str}"
            )
            start_date = date.fromisoformat(start_date_str)
            end_date = date.fromisoformat(end_date_str)
        else:
            # Use CLI semantics (like energy_mileage_daily)
            args = parse_arguments()
            # override push_to_db if --no-push used
            if args.no_push:
                push_to_db = False

            # optional ids from CLI
            if args.ids:
                ids = args.ids

            target_date = get_target_date(args)
            start_date = end_date = target_date

        # Single call to our range runner (it already loops day-by-day & id-by-id)
        logging.info(
            f"▶️ Starting TMS sessions job for {start_date} → {end_date} "
            f"(push_to_db={push_to_db})"
        )

        run_tms_sessions_range_to_iceberg(
            conn=conn,
            start_date=start_date,
            end_date=end_date,
            core_cols=CORE_COLS,
            df_mapping=None,  # or your mapping df when you have it
            source_schema=source_schema,
            source_table=source_table,
            target_schema=target_schema,
            target_table=target_table,
            partition_by=partition_by,
            push_to_db=push_to_db,
            collect_sessions=False,
            ids=ids,
        )

    except Exception as e:
        logging.critical(f"❌ A critical error occurred in TMS main: {e}", exc_info=True)
    finally:
        logging.info("🔒 STEP 5: Closing Trino connection...")
        conn.close()
        logging.info("✅ STEP 5: Connection closed.")


if __name__ == "__main__":
    # Airflow-style call (explicit range, DB write on):
    main(start_date_str="2025-09-01", end_date_str="2025-09-03", push_to_db=True)

    # CLI-style default: uses parse_arguments() + get_target_date()
    # main()